This notebook reads our DC2 synth model and saves the CSAMPLES file to 

/e/ocean1/users/vargatn/LSST/SYNTHETIC

In [12]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp


import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import synthetic.tools as tools
import synthetic.emulator.emulator as emulator
import synthetic.emulator.indexer as indexer
import synthetic.emulator.reader as reader

In [5]:
tag_root = "dc2-alpha_concentric_sample-v01_test-03"
NREPEATS = 4
NSAMPLES = 1600000
NCHUNKS = 160
bandwidth=0.1

root_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/"
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

LOGR_DRAW_RMINS = np.array([-3, -0.5, 0., 0.5])
LOGR_DRAW_RMAXS = np.array([-0.5, 0., 0.5, 1.2])
LOGR_CAT_RMAXS = [0., 0.5, 1.1, 1.2]

deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass")        
    ],
    "logs": [False, True, False, False, False, False, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16)],
}

wide_cr_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass"),
        ("LOGR", "R"),        
    ],
    "logs": [False, True, False, False, False, False, True, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16), (1e-3, 16)],
}

columns = {
    "cols_dc": ["COLOR_G_R", "COLOR_R_I",],
    "cols_wr": ["LOGR",],
    "cols_wcr": ["COLOR_G_R", "COLOR_R_I", "LOGR",],
}


In [6]:
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

In [7]:
refpixel = pd.read_hdf(deep_data_path, key="data")
table = pd.read_hdf(wide_data_path, key="data")

In [8]:
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_rands = emulator.construct_deep_container(refpixel, tmp_wide_cr_settings)

# loading deep catalogs
_deep_smc_settings = emulator.construct_deep_container(refpixel, deep_smc_settings)

# loading cluster data
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_clust = emulator.construct_deep_container(table, tmp_wide_cr_settings)


(1513572, 9)
(1513572,)
(1513572, 8)
(1513572,)
(1009658, 9)
(1009658,)


In [9]:
samples = []
scores = []
for rbin in np.arange(4):
    print(rbin)
#     expr = "/e/ocean1/users/vargatn/EMULATOR/EPSILON/resamples/epsilon_concentric_sample_v06_run*_rbin" + str(rbin) + "*samples.fits" 
    expr = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/dc2-alpha_concentric_sample-v01_test-03/dc2-alpha_concentric_sample-v01_test-03_run0*_rbin" + str(rbin) + "*samples.fits" 

    fnames_samples = np.sort(glob.glob(expr))
    fnames_scores = []
    for fname in fnames_samples:
        fnames_scores.append(fname.replace("samples.fits", "scores.fits"))

    samples_sep = []
    scores_sep = []
    for i, fname in enumerate(fnames_samples):
#         print(fname)
        samples_sep.append(fio.read(fname))
        scores_sep.append(fio.read(fnames_scores[i]))
        
    samples.append(np.hstack(samples_sep))
    scores.append(np.hstack(scores_sep))

0
1
2
3


In [10]:


mag_lims = (17, 22.5)
r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])



In [11]:


magcol = "mag_i"
ii = ((table[magcol] > mag_lims[0]) & (table[magcol] < mag_lims[1]))

clust_los_nums = np.histogram(np.log10(table[ii]["R"]), bins=redges)[0] / 41 # / nc

ii = ((refpixel[magcol] > mag_lims[0]) & (refpixel[magcol] < mag_lims[1]))
surfdens = len(refpixel[ii]) / hp.nside2pixarea(32, degrees=True) / 3600 / 3
rands_los_nums = surfdens * rareas
# rands_los_nums = np.histogram(np.log10(refpixel[ii]["R"]), bins=redges)[0] / rareas#* ratio# / nr
nratios = clust_los_nums / rands_los_nums



/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [13]:


ifields2 = []
iclusts2 = []
i2ds2 = []
for rbin in np.arange(4):
    print(rbin)
    _ifield, _iclust, _i2d = reader.result_reader2(samples[rbin], scores[rbin], nratio=nratios[rbin], m_factor=100, seed=rbin)
    ifields2.append(_ifield)
    iclusts2.append(_iclust)
    i2ds2.append(_i2d)



0
2.741989144315195
1
1.7913041757345514
2
1.1243442810539395
3
0.793271619957704


 In this exercise we only draw cluster samples, as the clusters will be injected to DC2

In [14]:


csamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][iclusts2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _csample = kde.random_draw(4e6)
    csamples.append(_csample)



0
1
2
3


In [15]:
csamples

[             GABS      SIZE      MAG_I  COLOR_G_R  COLOR_R_I  COLOR_I_Z  \
 0        0.320640 -0.901581  23.695770   1.213632   0.284145   0.148627   
 1        0.514918 -0.335548  23.276278   1.323697   0.550209   0.301413   
 2        0.309384 -0.376334  23.030974   0.539342   0.778464   0.504956   
 3        0.102149 -1.080650  24.798849   1.341419   0.538342   0.230514   
 4        0.639441 -0.350043  23.304387   0.936507   0.317449   0.202868   
 ...           ...       ...        ...        ...        ...        ...   
 3999995  0.107224 -0.507890  21.922076   1.322356   0.535423   0.282897   
 3999996  0.302257 -1.063861  24.180173  -0.430800   0.241704   0.103222   
 3999997  0.042797 -0.061218  20.395439   1.393519   0.650794   0.309385   
 3999998  0.126028 -0.354947  24.281529   1.149676   0.248537   0.114426   
 3999999  0.469052 -0.468760  24.352034   0.799514   1.198729   0.736513   
 
          STELLAR_MASS  HALO_MASS      LOGR  
 0            8.384907  11.027811 -0.661

In [20]:
opath = "/e/ocean1/users/vargatn/LSST/SYNTHETIC/csamples_nov_dev_01.h5"

In [32]:
reload(tools)

<module 'synthetic.tools' from '/home/moon/vargatn/DES/SYNTHETIC/synthetic/synthetic/tools.py'>

In [33]:
tools.save(opath, csamples)

In [29]:
tmp = load(opath)